# Module 5 Lab 1 - Accessing data via FHIR
[FHIR](https://www.hl7.org/fhir/) is a standard to provide an EMR agnostic interface to data contained in an electronic medical record.  The major EMR systems such as Epic and Cerner have provided FHIR api access to their systems, and as of 2020 the ONC has mandated a portion of the FHIR interface (the bulk data API) be part of any certified EHR.  

FHIR can be used to develop clinical decision support modules that can run on any EMR platform, in conjunction with [SMART](https://smarthealthit.org/).  SMART provides the framework for embedding an app into an EMR.  Apps can be developed by anyone, and when based on the FHIR API, can be deployed to any system that support the FHIR standard.

If you have developed a predictive model for patients, then the dissemination of that model can be done via a "SMART on FHIR" app.

Development of web-based SMART applications is beyond the scope of this course and usually beyond the scope of what a data scientist would do, but accessing data through a FHIR interface from an EHR is a relevant topic that we will explore in this module.

## fhirclient

We are going to use the fhirclient python package to interface to a public, fake patient database.
* The source code is available here: https://github.com/smart-on-fhir/client-py
* The API documentation is here: http://docs.smarthealthit.org/client-py/
* A list of FHIR servers is here: https://confluence.hl7.org/display/FHIR/Public+Test+Servers
* An online patient broswer for the server we will be using below is here: https://patient-browser.smarthealthit.org/?config=r3#/
* This lab is based on FHIR r3, documented here: http://hl7.org/fhir/STU3/


In [1]:
import sys
!{sys.executable} -m pip install fhirclient
from fhirclient import client

     |████████████████████████████████| 552kB 3.4MB/s eta 0:00:01
     |████████████████████████████████| 51kB 7.1MB/s  eta 0:00:01


## Retrieve a patient
First we will retrieve a patient by their patient identifier, which I found by browsing the online patient browser linked above.  This is the patient we will use for the next few examples: 

`_id=smart-1213208`  

You can find our patient [here](https://patient-browser.smarthealthit.org/?config=r3#/) by choosing "Advanced Mode" in the upper right corner for search, and inputing the above search criteria.

The fhirclient package is very object oriented.  Each item retrieved by the package is an object that must be interpreted properly to be of use.  The `Patient.read` method returns a `fhirclient.models.patient.Patient` object.  To get data from it we use attributes and methods on that object.

For example, we can get the `birthDate` using an attribute, but priniting that reveals it is also an object of type `fhirclient.models.fhirdate.FHIRDate`.  To get a human readable form of the date, we have to use an attribute of that object, `isostring`.  We can also get a python `Date` object as well.

`patient.name` is an array of patient names.  This patient has one alias.  We access with array notation, to get the first (and only) alias.  Again we notice that it is an object of type `fhirclient.models.humanname.HumanName`.  We can call a method on the smart api to get a formatted version of the name using `smart.human_name()`.  We can also access individual components of the name.  Notice that the attribute `given` is also an array.  This accounts for cultures that give more than a first and middle name, which can commonly happen in Spanish, Arabic, and many other cultures or regions.  

Documentation on object types can be found here: http://docs.smarthealthit.org/client-py/annotated.html.  You can use the search function to find the class documentation. HINT: Only search on the last part of the fully qualified object type, i.e. `Patient`, in order to get results.

Finally, all objects inherit from `FHIRAbstractBase`, which includes a method to get the object as a json string using `as_json()`.

In [2]:
import fhirclient.models.patient as p

settings = {
    'app_id': 'my_web_app',
    'api_base': 'https://r3.smarthealthit.org/'
}
smart = client.FHIRClient(settings=settings)

patient = p.Patient.read('smart-1213208', smart.server)

print('\nFind out what object type this is:', patient)
print('\nFind out what object type this is:', patient.birthDate)
print('\nGet a datetime object from a date:', type(patient.birthDate.date), patient.birthDate.date)
print('\nPrint the ISO form of the birthdate using an attribute of the object:', patient.birthDate.isostring)

print('\nFind out what object type this is:', patient.identifier)
print('\nHow many patient identifiers are there?', len(patient.identifier))
print('\nThe patient identifier attributes:', ', '.join([i.value for i in patient.identifier]))
print('\nCompare to this patient identifier:', patient.id)

print('\nHow many patient names are there?', len(patient.name))
print('\nFind out the object type for the first name in the list:', patient.name[0])
print('\nUse a smart utility to format the name:', smart.human_name(patient.name[0]))
print('\nUse attributes on the HumanName object to get name components:', 
      '\nFamily name:      ', patient.name[0].family, 
      '\nFirst given name: ', patient.name[0].given[0], 
      '\nSecond given name:', patient.name[0].given[1], 
      '\nAll given names:  ', ' '.join(patient.name[0].given))

print()
print('as json:')
print(patient.as_json())


Find out what object type this is: <fhirclient.models.patient.Patient object at 0x7efd8d722d30>

Find out what object type this is: <fhirclient.models.fhirdate.FHIRDate object at 0x7efd8d6bda58>

Get a datetime object from a date: <class 'datetime.date'> 1968-12-15

Print the ISO form of the birthdate using an attribute of the object: 1968-12-15

Find out what object type this is: [<fhirclient.models.identifier.Identifier object at 0x7efd8d25ccc0>]

How many patient identifiers are there? 1

The patient identifier attributes: smart-1213208

Compare to this patient identifier: smart-1213208

How many patient names are there? 1

Find out the object type for the first name in the list: <fhirclient.models.humanname.HumanName object at 0x7efd8d25ce80>

Use a smart utility to format the name: Brian Q Gracia

Use attributes on the HumanName object to get name components: 
Family name:       Gracia 
First given name:  Brian 
Second given name: Q 
All given names:   Brian Q

as json:
{'id': 's

## Retrieve conditions
Here we will retrieve conditions that have been documented for the patient.  Each condition will be of type `fhirclient.models.condition.Condition`.  The text is a `Narrative` object. This kind of object is represented as HTML, and might look like: 
```<div xmlns="http://www.w3.org/1999/xhtml">Unspecified local infection of skin and subcutaneous tissue</div>```

It can include html `<img>` tags and other rich media.  We get the `div` attribute of that object to see the narrative.  We use a python package called `BeautifulSoup` to extract the text from the div. [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/) is an html parsing engine.

In [3]:
import fhirclient.models.condition as c
from bs4 import BeautifulSoup

search = c.Condition.where(struct={'patient': patient.id})

conditions = search.perform_resources(smart.server)
for cond in conditions:
    print(cond.onsetDateTime.isostring, ':', BeautifulSoup(cond.text.div).div.string)
    print(' raw html:', cond.text.div)
    print()


2012-11-03 : Unspecified local infection of skin and subcutaneous tissue
 raw html: <div xmlns="http://www.w3.org/1999/xhtml">Unspecified local infection of skin and subcutaneous tissue</div>

2012-11-07 : Joint effusion of the lower leg
 raw html: <div xmlns="http://www.w3.org/1999/xhtml">Joint effusion of the lower leg</div>

2012-11-05 : Cellulitis and abscess of leg
 raw html: <div xmlns="http://www.w3.org/1999/xhtml">Cellulitis and abscess of leg</div>

2012-11-07 : Lymphadenitis
 raw html: <div xmlns="http://www.w3.org/1999/xhtml">Lymphadenitis</div>

2012-11-05 : Gastro-esophageal reflux disease with esophagitis
 raw html: <div xmlns="http://www.w3.org/1999/xhtml">Gastro-esophageal reflux disease with esophagitis</div>

2012-11-07 : Abdominal pain
 raw html: <div xmlns="http://www.w3.org/1999/xhtml">Abdominal pain</div>

2012-12-18 : Cellulitis and abscess of face
 raw html: <div xmlns="http://www.w3.org/1999/xhtml">Cellulitis and abscess of face</div>

2012-08-12 : Neck pain
 r

## Accessing data as a bundle
We can also get our list of conditions as a `Bundle` object.  In fact, any search can be retrieved as a `Bundle`.  The `entry` attributes returns a list of `BundleEntry` objects.  The `fullUrl` attribute will give a link to retrieve the information from a web interface.  The `resource` attribute gives you the `Condition` object that we used above.

In [4]:
bundle = search.perform(smart.server)

print('Click here to get the details from a web interface:', bundle.entry[0].fullUrl)
print()

print("This returns the same div tag as above", bundle.entry[0].resource.text.div)

print()
print("All the patients conditions")
# loop through them all (the same information retrieved above but through a Bundle)
for e in bundle.entry:
    print(' ', e.resource.onsetDateTime.isostring, BeautifulSoup(e.resource.text.div).div.string)

Click here to get the details from a web interface: https://r3.smarthealthit.org/Condition/smart-Condition-155

This returns the same div tag as above <div xmlns="http://www.w3.org/1999/xhtml">Unspecified local infection of skin and subcutaneous tissue</div>

All the patients conditions
  2012-11-03 Unspecified local infection of skin and subcutaneous tissue
  2012-11-07 Joint effusion of the lower leg
  2012-11-05 Cellulitis and abscess of leg
  2012-11-07 Lymphadenitis
  2012-11-05 Gastro-esophageal reflux disease with esophagitis
  2012-11-07 Abdominal pain
  2012-12-18 Cellulitis and abscess of face
  2012-08-12 Neck pain
  2012-08-12 Spasm
  2012-11-05 Benign essential hypertension


## Retrieve procedures
Retrieving procedures is similar, but with different objects.  A [`Procedure`](https://docs.smarthealthit.org/client-py/classfhirclient_1_1models_1_1procedure_1_1_procedure.html) object has a `code` attribute that is a [`CodeableConcept`](https://docs.smarthealthit.org/client-py/classfhirclient_1_1models_1_1codeableconcept_1_1_codeable_concept.html) object, which has a `coding` attribute, which is an array of [`Coding`](https://docs.smarthealthit.org/client-py/classfhirclient_1_1models_1_1coding_1_1_coding.html) objects.

We are using the `where` method of the `Procedure` object to query for our patient's procedures.  This takes a python dictionary keyed by the attributes we want to search, and the values specify the values we want to search by.

Refer to the documentation to see the attributes that are available for each of these classes.

We are using this patient's conditions: `_id=7b697322-3607-46cb-a240-c081bccba2e5`

Note that in the search, we specify the patient identifier so we only get encounters for that patient, along with the statuses that we want.  Note that the statuses are comma separated.  

In [5]:
import fhirclient.models.procedure as p

search = p.Procedure.where(struct={'patient': '7b697322-3607-46cb-a240-c081bccba2e5', 'status': 'completed,active'})
procedures = search.perform_resources(smart.server)
for procedure in procedures:
    print(procedure.code.text)
    print(' ', procedure.status, procedure.code.coding[0].system, procedure.code.coding[0].code, procedure.code.coding[0].display)


Documentation of current medications
  completed http://snomed.info/sct 428191000124101 Documentation of current medications
Surgical manipulation of joint of knee
  completed http://snomed.info/sct 699253003 Surgical manipulation of joint of knee
Measurement of respiratory function (procedure)
  completed http://snomed.info/sct 23426006 Measurement of respiratory function (procedure)
Documentation of current medications
  completed http://snomed.info/sct 428191000124101 Documentation of current medications


## Retrieve encounters
Encounters describe when the clinic or hospital saw a patient.  They can be outpatient, inpatient, emergency, etc.  This is the same basic approach as we used to get procedures, but with new objects.

`type` is an array of `CodeableConcept`, which we saw above.

`period` is a `Period` object which describes the `start` and `end` of the encounter.  Search for this class definition here: https://docs.smarthealthit.org/client-py/annotated.html

`reason` is an array of `CodeableConcept`.  Here we can see that we have to handle instances where an encounter may not have a reason entered.


In [6]:
import fhirclient.models.encounter as e

search = e.Encounter.where(struct={'patient': '7b697322-3607-46cb-a240-c081bccba2e5'})
encntrs = search.perform_resources(smart.server)
for encntr in encntrs:
    print()
    if encntr.reason:
        print(encntr.reason[0].coding[0].display, encntr.reason[0].coding[0].system, encntr.reason[0].coding[0].code)
    else:
        print('No reason provided')
    print(' ', 'status:', encntr.status, 'start:', encntr.period.start.isostring, 'end:', encntr.period.end.isostring)
    print(' ', encntr.type[0].coding[0].system, encntr.type[0].text, encntr.type[0].coding[0].code)



Contact dermatitis http://snomed.info/sct 40275004
  status: finished start: 2014-05-20T17:14:09Z end: 2014-05-20T17:14:09Z
  http://snomed.info/sct Encounter for problem 185347001

No reason provided
  status: finished start: 2014-05-22T18:20:14Z end: 2014-05-22T18:20:14Z
  http://snomed.info/sct Encounter for problem 185347001

No reason provided
  status: finished start: 2018-04-02T22:25:55Z end: 2018-04-02T23:25:55Z
  http://snomed.info/sct Outpatient Encounter 185349003

No reason provided
  status: finished start: 2013-11-16T02:31:37Z end: 2013-11-16T02:31:37Z
  http://snomed.info/sct Emergency room admission 50849002

Acute bronchitis (disorder) http://snomed.info/sct 10509002
  status: finished start: 2013-12-29T08:08:01Z end: 2013-12-29T08:24:56Z
  http://snomed.info/sct Encounter for symptom 185345009

Viral sinusitis (disorder) http://snomed.info/sct 444814009
  status: finished start: 2011-01-17T23:07:27Z end: 2011-01-17T23:07:27Z
  http://snomed.info/sct Encounter for sym

## Conditional searches
Resources can be returned using less than and greater than, plus other conditional logic.  The search structure is a little bit different, and is similar to the search structure used for NoSQL databases like MongoDB.

These are the operators that are supported:

|term|description|
|---|---|
| `$eq` | Match the values that are equal to a specified value. |
| `$gt` | Match the values that are greater than a specified value. |
| `$gte` | Match all the values that are greater than or equal to a specified value. |
| `$lt` | Match all the values that are less than a specified value. |
| `$lte` | Match all the values that are less than or equal to a specified value. |
| `$ne` | Match all the values that are not equal to a specified value. |

We will search encounters for the above patient that are newer than January 1, 2015.  We also introduce a new method of searching by a reference id, using the `subject` parameter rather than specifying the `patient` parameter.  This will search for any reference object that matches the passed identifier (which may not always be a patient id).

Date and date times are passed as strings, and must be in the following format:

`yyyy-mm-ddThh:mm:ss[Z|(+|-)hh:mm]`

See [here](https://www.hl7.org/fhir/search.html#date) for more info on date time formats, paying attention to the following: "Any degree of precision can be provided, but it SHALL be populated from the left (e.g. can't specify a month without a year), except that the minutes SHALL be present if an hour is present, and you SHOULD provide a time zone if the time part is present. Note: Time can consist of hours and minutes with no seconds")

If a mistake is made in the query, you will receive an error such as the following:

```
HTTPError: 400 Client Error: Bad Request for url: https://r3.smarthealthit.org/Encounter?patient=7b697322-3607-46cb-a240-c081bccba2e5&Date=%3E2015-01-01
```
Clicking on that link will provide more information on the cause of the error. (In this case, the `date` search term was mistakenly capitalized.  Search terms are case sensitive)

Search parameters are documented here:
http://hl7.org/fhir/searchparameter-registry.html

In [7]:
search = e.Encounter.where(struct={'subject': '7b697322-3607-46cb-a240-c081bccba2e5', 
                                   'date': {'$gt': '2015-01-01'}})
encntrs = search.perform_resources(smart.server)
for encntr in encntrs:
    print()
    if encntr.reason:
        print(encntr.reason[0].coding[0].display, encntr.reason[0].coding[0].system, encntr.reason[0].coding[0].code)
    else:
        print('No reason provided')
    print(' ', 'status:', encntr.status, 'start:', encntr.period.start.isostring, 'end:', encntr.period.end.isostring)
    print(' ', encntr.type[0].coding[0].system, encntr.type[0].text, encntr.type[0].coding[0].code)



No reason provided
  status: finished start: 2018-04-02T22:25:55Z end: 2018-04-02T23:25:55Z
  http://snomed.info/sct Outpatient Encounter 185349003

Viral sinusitis (disorder) http://snomed.info/sct 444814009
  status: finished start: 2018-05-11T20:11:44Z end: 2018-05-11T20:11:44Z
  http://snomed.info/sct Encounter for symptom 185345009

Sinusitis (disorder) http://snomed.info/sct 36971009
  status: finished start: 2016-06-26T05:43:13Z end: 2018-05-11T20:11:44Z
  http://snomed.info/sct Encounter for symptom 185345009

No reason provided
  status: finished start: 2015-03-26T13:59:51Z end: 2015-03-26T14:59:51Z
  http://snomed.info/sct Outpatient Encounter 185349003

Sinusitis (disorder) http://snomed.info/sct 36971009
  status: finished start: 2016-04-07T21:44:16Z end: 2016-04-07T21:44:16Z
  http://snomed.info/sct Encounter for symptom 185345009

Sinusitis (disorder) http://snomed.info/sct 36971009
  status: finished start: 2016-06-19T05:43:13Z end: 2016-06-19T05:43:13Z
  http://snomed.

## Additional data available
There are many other types of data accessible through a FHIR interface.  All data access would follow a similar path as this, using different types of objects along the way, which can be looked up in the [online documentation](http://docs.smarthealthit.org/client-py/).